In [1]:
# !pip install pandas
# !pip install openpyxl

In [2]:
import pandas as pd
import glob

## 최신 법정동 코드 DF

- 법정동명이 '서울특별시 OO구'인 것만 남기고, 'OO구'로 변환
- 딕셔너리로 생성 : {'법정동명':법정동코드}

In [3]:
new_code_df = pd.read_excel('../dedev_2nd_project/법정동코드 조회자료.xlsx')
new_code_df.head()

,법정동코드,법정동명
0,1100000000,서울특별시
1,1111000000,서울특별시 종로구
2,1111010100,서울특별시 종로구 청운동
3,1111010200,서울특별시 종로구 신교동
4,1111010300,서울특별시 종로구 궁정동


In [4]:
filtered_gucode_df = new_code_df[new_code_df['법정동명'].str.endswith('구')]
new_code_dic = {row['법정동명'].split()[-1]: row['법정동코드'] for _, row in filtered_gucode_df.iterrows()}
print(new_code_dic)
print('서울특별시의 자치구 수 :', len(new_code_dic))

{'종로구': 1111000000, '중구': 1114000000, '용산구': 1117000000, '성동구': 1120000000, '광진구': 1121500000, '동대문구': 1123000000, '중랑구': 1126000000, '성북구': 1129000000, '강북구': 1130500000, '도봉구': 1132000000, '노원구': 1135000000, '은평구': 1138000000, '서대문구': 1141000000, '마포구': 1144000000, '양천구': 1147000000, '강서구': 1150000000, '구로구': 1153000000, '금천구': 1154500000, '영등포구': 1156000000, '동작구': 1159000000, '관악구': 1162000000, '서초구': 1165000000, '강남구': 1168000000, '송파구': 1171000000, '강동구': 1174000000}
서울특별시의 자치구 수 : 25


## 행정동단위 29개 통신정보 전처리

### 데이터 읽어오기

In [5]:
file_list = glob.glob('../dedev_2nd_project/23_commuinfo_lifedata/*.xlsx')
all_commuinfo_life_df = []

for m, file_path in enumerate(file_list):
    df = pd.read_excel(file_path)
    if m + 1 >= 10:
        df['month'] = str(m+1)
    else:
        df['month'] = '0' + str(m+1)
    all_commuinfo_life_df.append(df)

commuinfo_life_df = pd.concat(all_commuinfo_life_df, ignore_index=True)

### 1.컬럼 삭제
    - 4분위수, 미추정 컬럼들 제외

In [6]:
select_col = [
    'month', '행정동코드', '자치구', '성별', '연령대', '총인구수', '1인가구수', '야간상주지 변경횟수 평균', '주간상주지 변경횟수 평균',
    '평균 출근 소요시간 평균', '평균 근무시간 평균', '소액결재 사용횟수 평균', '소액결재 사용금액 평균', '최근 3개월 내 요금 연체 비율',
    '카카오톡 비사용 인구수', 'SNS 사용횟수', '평균 통화량', '평균 문자량', '평균 통화대상자 수', '평균 문자대상자 수', '데이터 사용량',
    '평일 총 이동 횟수', '휴일 총 이동 횟수 평균', '집 추정 위치 평일 총 체류시간', '집 추정 위치 휴일 총 체류시간', '평일 총 이동 거리 합계',
    '휴일 총 이동 거리 합계', '지하철이동일수 합계', '게임 서비스 사용일수', '금융 서비스 사용일수', '쇼핑 서비스 사용일수', '동영상/방송 서비스 사용일수',
    '유튜브 사용일수', '넷플릭스 사용일수', '배달 서비스 사용일수', '배달_브랜드 서비스 사용일수', '배달_식재료 서비스 사용일수']
del_commuinfo_life_df = commuinfo_life_df[select_col]
del_commuinfo_life_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91584 entries, 0 to 91583
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   month              91584 non-null  object 
 1   행정동코드              91584 non-null  int64  
 2   자치구                91584 non-null  object 
 3   성별                 91584 non-null  int64  
 4   연령대                91584 non-null  int64  
 5   총인구수               91584 non-null  float64
 6   1인가구수              91584 non-null  float64
 7   야간상주지 변경횟수 평균      91584 non-null  float64
 8   주간상주지 변경횟수 평균      91584 non-null  float64
 9   평균 출근 소요시간 평균      91584 non-null  float64
 10  평균 근무시간 평균         91584 non-null  float64
 11  소액결재 사용횟수 평균       91584 non-null  float64
 12  소액결재 사용금액 평균       91584 non-null  float64
 13  최근 3개월 내 요금 연체 비율  91584 non-null  float64
 14  카카오톡 비사용 인구수       91584 non-null  float64
 15  SNS 사용횟수           91584 non-null  float64
 16  평균 통화량             915

### 2. 법정동코드 최신 코드로 변경
    - 코드값 변경
    - 자치구 컬럼 삭제

In [7]:
for key, val in new_code_dic.items():
    del_commuinfo_life_df.loc[del_commuinfo_life_df['자치구'] == key, '행정동코드'] = val

del_gu_df = del_commuinfo_life_df.drop('자치구', axis=1)
del_gu_df.head()

,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,1,20,220.00,68.91,2.57,4.06,89.09,536.34,...,8.38,74.16,96.40,108.96,39.24,0.24,-0.18,14.34,2.71,1.67
1,01,1111000000,1,25,321.00,111.76,2.30,3.23,52.00,580.46,...,10.71,71.15,137.81,157.28,46.97,0.22,-0.15,18.88,2.82,1.67
2,01,1111000000,1,30,350.23,112.44,2.02,2.96,43.93,522.67,...,9.93,61.57,149.82,178.59,47.49,0.13,-0.18,20.59,2.85,1.67
3,01,1111000000,1,35,353.00,111.49,1.87,2.56,62.00,556.96,...,10.21,57.91,147.34,180.28,46.21,0.05,-0.18,18.32,3.46,1.67
4,01,1111000000,1,40,419.00,77.91,1.66,2.47,68.21,546.73,...,9.31,54.41,143.42,185.52,44.21,-0.02,-0.23,17.76,2.50,1.67


### 3. groupby
    - 행정동코드, 성별, 연령대, month

In [8]:
group_commuinfo_life_df = del_gu_df.groupby(['month', '행정동코드', '성별', '연령대']).mean().reset_index()
group_commuinfo_life_df.head()

,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,1,20,403.294118,156.362941,2.573529,3.813529,79.797059,508.210588,...,8.198235,83.579412,108.312941,129.602353,41.534118,0.290588,-0.139412,17.748824,2.640000,0.785882
1,01,1111000000,1,25,436.470588,166.838235,2.123529,3.515882,63.098235,535.671176,...,9.658824,72.841176,125.212353,157.306471,45.659412,0.254118,-0.196471,19.557647,2.815882,1.178824
2,01,1111000000,1,30,382.013529,131.118235,1.950000,2.827647,66.531176,555.128235,...,10.195294,68.944706,143.054118,179.457059,47.483529,0.178235,-0.185882,21.341176,2.726471,1.277059
3,01,1111000000,1,35,324.647059,98.771765,1.836471,2.623529,65.673529,551.735294,...,9.987647,66.612353,144.312353,190.807647,46.211765,0.088235,-0.213529,18.988824,3.076471,1.080588
4,01,1111000000,1,40,380.235294,70.197647,1.741176,2.554118,68.120000,555.632353,...,9.754706,63.718235,137.843529,186.182941,42.975294,-0.002941,-0.291765,16.975882,2.857059,1.375294


### 4. 성별
    - 남자(1) -> 0
    - 여자(2) -> 1

In [9]:
group_commuinfo_life_df['성별'] = group_commuinfo_life_df['성별'].replace({1:0, 2:1})
group_commuinfo_life_df.head()

,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,0,20,403.294118,156.362941,2.573529,3.813529,79.797059,508.210588,...,8.198235,83.579412,108.312941,129.602353,41.534118,0.290588,-0.139412,17.748824,2.640000,0.785882
1,01,1111000000,0,25,436.470588,166.838235,2.123529,3.515882,63.098235,535.671176,...,9.658824,72.841176,125.212353,157.306471,45.659412,0.254118,-0.196471,19.557647,2.815882,1.178824
2,01,1111000000,0,30,382.013529,131.118235,1.950000,2.827647,66.531176,555.128235,...,10.195294,68.944706,143.054118,179.457059,47.483529,0.178235,-0.185882,21.341176,2.726471,1.277059
3,01,1111000000,0,35,324.647059,98.771765,1.836471,2.623529,65.673529,551.735294,...,9.987647,66.612353,144.312353,190.807647,46.211765,0.088235,-0.213529,18.988824,3.076471,1.080588
4,01,1111000000,0,40,380.235294,70.197647,1.741176,2.554118,68.120000,555.632353,...,9.754706,63.718235,137.843529,186.182941,42.975294,-0.002941,-0.291765,16.975882,2.857059,1.375294


### 5. 연령대 : 20~60대(10대 간격)
    - 연령대 컬럼의 값이 일의자리가 5인 것을 0으로 변경
    - groupby : month, 행정동코드, 성별, 연령대가 같은 것을 Sum
    - 70대 삭제

In [10]:
group_commuinfo_life_df['연령대'] = group_commuinfo_life_df['연령대'] // 10 * 10
age_commuinfo_life_df = group_commuinfo_life_df.groupby(['month', '행정동코드', '성별', '연령대']).sum().reset_index()
age_commuinfo_life_df.head()

,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,0,20,839.764706,323.201176,4.697059,7.329412,142.895294,1043.881765,...,17.857059,156.420588,233.525294,286.908824,87.193529,0.544706,-0.335882,37.306471,5.455882,1.964706
1,01,1111000000,0,30,706.660588,229.890000,3.786471,5.451176,132.204706,1106.863529,...,20.182941,135.557059,287.366471,370.264706,93.695294,0.266471,-0.399412,40.330000,5.802941,2.357647
2,01,1111000000,0,40,765.588235,141.277059,3.479412,4.962353,135.382353,1105.467647,...,19.786471,129.578824,266.130000,363.095294,84.100000,-0.020000,-0.631765,32.160000,5.914118,2.455882
3,01,1111000000,0,50,759.117647,118.685882,3.231176,4.730588,134.302353,1080.358235,...,18.637059,104.087647,187.772353,256.898824,64.977647,-0.158235,-0.940000,24.310000,5.742941,1.670000
4,01,1111000000,0,60,463.941176,95.472353,3.111176,4.708824,134.247059,1039.260588,...,17.159412,85.901765,110.528824,157.657647,47.144706,-0.272941,-1.180000,18.202353,2.942353,1.375294


In [11]:
del_age_commuinfo_life_df = age_commuinfo_life_df[age_commuinfo_life_df['연령대'] != 70]
del_age_commuinfo_life_df.head(10)

,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,0,20,839.764706,323.201176,4.697059,7.329412,142.895294,1043.881765,...,17.857059,156.420588,233.525294,286.908824,87.193529,0.544706,-0.335882,37.306471,5.455882,1.964706
1,01,1111000000,0,30,706.660588,229.890000,3.786471,5.451176,132.204706,1106.863529,...,20.182941,135.557059,287.366471,370.264706,93.695294,0.266471,-0.399412,40.330000,5.802941,2.357647
2,01,1111000000,0,40,765.588235,141.277059,3.479412,4.962353,135.382353,1105.467647,...,19.786471,129.578824,266.130000,363.095294,84.100000,-0.020000,-0.631765,32.160000,5.914118,2.455882
3,01,1111000000,0,50,759.117647,118.685882,3.231176,4.730588,134.302353,1080.358235,...,18.637059,104.087647,187.772353,256.898824,64.977647,-0.158235,-0.940000,24.310000,5.742941,1.670000
4,01,1111000000,0,60,463.941176,95.472353,3.111176,4.708824,134.247059,1039.260588,...,17.159412,85.901765,110.528824,157.657647,47.144706,-0.272941,-1.180000,18.202353,2.942353,1.375294
6,01,1111000000,1,20,840.874706,335.627647,4.034706,6.215882,121.560588,1056.348235,...,18.676471,83.310000,219.532353,316.589412,71.593529,0.254706,-0.291765,34.452941,4.645882,3.143529
7,01,1111000000,1,30,662.352941,199.273529,3.520000,5.448824,111.594706,1056.054706,...,17.021765,89.384706,266.319412,424.563529,82.898824,0.047647,-0.402353,36.034706,5.623529,3.143529
8,01,1111000000,1,40,787.352941,134.149412,3.190000,4.922353,117.398824,1030.108235,...,15.111765,94.200588,240.380000,416.971765,74.774706,-0.132941,-0.624706,31.108235,5.515882,2.947059
9,01,1111000000,1,50,782.309412,116.473529,3.092353,4.882353,120.915294,1041.767059,...,14.414118,84.323529,168.928235,293.455294,50.601765,-0.285882,-0.941176,21.954118,5.134706,2.554118
10,01,1111000000,1,60,514.941176,105.820000,3.081176,5.013529,123.247647,968.091765,...,12.927647,78.939412,92.738824,167.641176,36.065294,-0.376471,-1.180000,20.008824,1.521176,1.866471


### 6. 총인구수, 1인가구수
    - 소수부분 버림

In [12]:
del_age_commuinfo_life_df['총인구수'] = del_age_commuinfo_life_df['총인구수'].astype(int)
del_age_commuinfo_life_df['1인가구수'] = del_age_commuinfo_life_df['1인가구수'].astype(int)
del_age_commuinfo_life_df.iloc[:, 6:] = del_age_commuinfo_life_df.iloc[:, 6:].round(2)
final_commuinfo_life_df = del_age_commuinfo_life_df
final_commuinfo_life_df.head()

/tmp/ipykernel_67180/4231603158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del_age_commuinfo_life_df['총인구수'] = del_age_commuinfo_life_df['총인구수'].astype(int)
/tmp/ipykernel_67180/4231603158.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del_age_commuinfo_life_df['1인가구수'] = del_age_commuinfo_life_df['1인가구수'].astype(int)


,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,0,20,839,323,4.70,7.33,142.90,1043.88,...,17.86,156.42,233.53,286.91,87.19,0.54,-0.34,37.31,5.46,1.96
1,01,1111000000,0,30,706,229,3.79,5.45,132.20,1106.86,...,20.18,135.56,287.37,370.26,93.70,0.27,-0.40,40.33,5.80,2.36
2,01,1111000000,0,40,765,141,3.48,4.96,135.38,1105.47,...,19.79,129.58,266.13,363.10,84.10,-0.02,-0.63,32.16,5.91,2.46
3,01,1111000000,0,50,759,118,3.23,4.73,134.30,1080.36,...,18.64,104.09,187.77,256.90,64.98,-0.16,-0.94,24.31,5.74,1.67
4,01,1111000000,0,60,463,95,3.11,4.71,134.25,1039.26,...,17.16,85.90,110.53,157.66,47.14,-0.27,-1.18,18.20,2.94,1.38


### 7. 총인구수 컬럼 변경
    - interest_life와 동일하게 총인구로 컬럼명 변경

In [13]:
final_commuinfo_life_df = final_commuinfo_life_df.rename(columns={'총인구수':'총인구'})
final_commuinfo_life_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2250 entries, 0 to 2698
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   month              2250 non-null   object 
 1   행정동코드              2250 non-null   int64  
 2   성별                 2250 non-null   int64  
 3   연령대                2250 non-null   int64  
 4   총인구                2250 non-null   int64  
 5   1인가구수              2250 non-null   int64  
 6   야간상주지 변경횟수 평균      2250 non-null   float64
 7   주간상주지 변경횟수 평균      2250 non-null   float64
 8   평균 출근 소요시간 평균      2250 non-null   float64
 9   평균 근무시간 평균         2250 non-null   float64
 10  소액결재 사용횟수 평균       2250 non-null   float64
 11  소액결재 사용금액 평균       2250 non-null   float64
 12  최근 3개월 내 요금 연체 비율  2250 non-null   float64
 13  카카오톡 비사용 인구수       2250 non-null   float64
 14  SNS 사용횟수           2250 non-null   float64
 15  평균 통화량             2250 non-null   float64
 16  평균 문자량             2250 non-n

### CSV 저장

In [14]:
final_commuinfo_life_df.to_csv('./pre_data/pre_commuinfo_life.csv', index=False)